# 07 – Generalized Model (Cross-Player, Robustness Test)

In this notebook, we aim to train a more general model that works across different players and shooting styles. This involves:
- Aggregating data from multiple players
- Normalizing poses to account for size, angle, or camera variation
- Evaluating whether the model generalizes well to unseen players
- Discussing robustness and keypoints most predictive of shot direction

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking

# Supposé : X, y déjà préchargés (concaténation multi-joueurs)
# Labels en g/m/d et fichier penalty_labels.csv avec colonne 'player' ajoutée

In [ ]:
# normaliser chaque séquence individuellement (centrer autour du bassin)
def normalize_sequence(seq):
    seq = np.array(seq)
    if seq.shape[1] % 2 != 0:
        return seq
    pelvis_x = seq[:, 0]
    pelvis_y = seq[:, 1]
    coords = seq.reshape(seq.shape[0], -1, 2)
    coords[:, :, 0] -= pelvis_x[:, None]
    coords[:, :, 1] -= pelvis_y[:, None]
    return coords.reshape(seq.shape[0], -1)

X = np.array([normalize_sequence(s) for s in X])

In [ ]:
# Encodage et split
label_map = {'g': 0, 'm': 1, 'd': 2}
y_cat = to_categorical([label_map[l] for l in y], num_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

In [ ]:
# Entraînement modèle LSTM généralisé
model = Sequential([
    Masking(mask_value=0., input_shape=(X.shape[1], X.shape[2])),
    LSTM(64),
    Dropout(0.4),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=25, batch_size=16, validation_data=(X_test, y_test))

In [ ]:
# Évaluation finale du modèle généralisé
y_pred = model.predict(X_test)
y_true = np.argmax(y_test, axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_true, y_pred_classes, target_names=['g', 'm', 'd']))